In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
import tensorflow as tf
from datetime import datetime

In [2]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [3]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)# Load the data

In [4]:
# Load the data
data_root_path = '/Users/zhengyixing/Documents/study/3nd_semester/DL/homework/cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')
indexs = np.random.choice(50000, 5000, replace=False)
X_validation = X_train[:,indexs].T
y_validation = y_train[indexs]
X_train = np.delete(X_train,indexs, axis = 1).T
y_train = np.delete(y_train,indexs)

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [5]:
def get_batch(X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        m = X.shape[1]
        start_index = np.random.randint(0, m - batch_size)
        X_batch = X[start_index:(start_index + batch_size), :]
        y_batch = y[start_index:(start_index + batch_size)]
        
        return X_batch, y_batch

In [6]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)
epoch = 50
batch_size = 128
learning_rate = 0.001

with tf.name_scope('input') as scope:
    X = tf.placeholder(tf.float32, shape = (None, 3072))
    y = tf.placeholder(tf.int32, shape = (None))
    dropout_rate = tf.placeholder(tf.float32, shape=())
    training = tf.placeholder(tf.bool)

In [7]:
input_layer = tf.reshape(X, [-1, 32, 32, 3])

with tf.name_scope('conv1') as scope:
    conv1 = tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=3, strides=1, 
                            padding = 'SAME', activation = tf.nn.relu, name='conv1')
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=2, padding='SAME', name='pool1')

with tf.name_scope('conv2') as scope:
    conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=3, strides=1, 
                        padding = 'SAME', activation = tf.nn.relu, name='conv2')
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=2, strides=2, padding='SAME', name='pool2')

with tf.name_scope('fc') as scope:
    pool2_flat = tf.reshape(pool2, [-1, 8*8*64], name='pool2_flat')
    dense = tf.layers.dense(pool2_flat, units = 8*64, activation = tf.nn.relu, name='dense')
    dropout = tf.layers.dropout(dense, rate=dropout_rate, training=training, name='dropout')
    
with tf.name_scope('logits') as scope:
    logits = tf.layers.dense(dropout, units = 10, name='logits')
    
with tf.name_scope('loss') as scope:
    softmax = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name = 'softmax')
    loss = tf.reduce_mean(softmax, name = 'loss')    

with tf.name_scope('train') as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
with tf.name_scope('eval') as scope:
    correct = tf.nn.in_top_k(logits, y, 1, name='correct')
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
with tf.name_scope('summary') as scope:
    loss_summary = tf.summary.scalar('LOSS', loss)
    accuracy_symmary = tf.summary.scalar('ACCURACY', accuracy)
    test_loss_summary = tf.summary.scalar('test_LOSS', loss)
    test_accuracy_symmary = tf.summary.scalar('test_ACCURACY', accuracy)
    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
    
with tf.name_scope('init') as scope:
    init = tf.global_variables_initializer()
    

In [8]:
def train(X_train, y_train, X_validation, y_validation, batch_size, epoch):
    training_num = X_train.shape[0]
    iteration = training_num // batch_size
    
    with tf.Session() as sess:
        sess.run(init)
        for i in range(epoch):
            test_loss_summary_str =  test_loss_summary.eval(feed_dict={X: X_validation, y:y_validation,
                                                             dropout_rate:0.3, training:False})
            test_accuracy_symmary_str = test_accuracy_symmary.eval(feed_dict={X: X_validation, y:y_validation,
                                                             dropout_rate:0.3, training:False})
            file_writer.add_summary(test_loss_summary_str, i)
            file_writer.add_summary(test_accuracy_symmary_str, i)
            
            for j in range(iteration):
                X_batch, y_batch = get_batch(X_train, y_train, batch_size)
                sess.run(optimizer, feed_dict={X: X_batch, y:y_batch, dropout_rate:0.3, training:True})
                step = j + i * iteration
                if step % 100 == 0:
                        loss_ = loss.eval(feed_dict={X: X_batch, y:y_batch, dropout_rate:0.3, training:False})
                        accuracy_ = accuracy.eval(feed_dict={X: X_batch, y:y_batch,
                                                             dropout_rate:0.3, training:False})
                        loss_summary_str =  loss_summary.eval(feed_dict={X: X_batch, y:y_batch,
                                                                          dropout_rate:0.3, training:False})
                        accuracy_symmary_str = accuracy_symmary.eval(feed_dict={X: X_batch, y:y_batch,
                                                             dropout_rate:0.3, training:False})
                        print('after '+ str(step) + ' iterations' + 
                              'the loss is ' + str(loss_) + ', the accuracy is ' + str(accuracy_))
                        file_writer.add_summary(loss_summary_str, step)
                        file_writer.add_summary(accuracy_symmary_str, step)
            

In [9]:
train(X_train, y_train, X_validation, y_validation, batch_size, epoch)

after 0 iterationsthe loss is 2.31124, the accuracy is 0.078125
after 100 iterationsthe loss is 2.29308, the accuracy is 0.15625
after 200 iterationsthe loss is 2.2992, the accuracy is 0.125
after 300 iterationsthe loss is 2.29343, the accuracy is 0.15625
after 400 iterationsthe loss is 2.28602, the accuracy is 0.179688
after 500 iterationsthe loss is 2.28324, the accuracy is 0.164062
after 600 iterationsthe loss is 2.28562, the accuracy is 0.171875
after 700 iterationsthe loss is 2.26635, the accuracy is 0.148438
after 800 iterationsthe loss is 2.25132, the accuracy is 0.164062
after 900 iterationsthe loss is 2.23672, the accuracy is 0.179688
after 1000 iterationsthe loss is 2.25197, the accuracy is 0.226562
after 1100 iterationsthe loss is 2.24228, the accuracy is 0.203125
after 1200 iterationsthe loss is 2.22087, the accuracy is 0.25
after 1300 iterationsthe loss is 2.24026, the accuracy is 0.242188
after 1400 iterationsthe loss is 2.20165, the accuracy is 0.28125
after 1500 iterati

after 12400 iterationsthe loss is 1.42128, the accuracy is 0.546875
after 12500 iterationsthe loss is 1.47796, the accuracy is 0.5
after 12600 iterationsthe loss is 1.52939, the accuracy is 0.492188
after 12700 iterationsthe loss is 1.43035, the accuracy is 0.476562
after 12800 iterationsthe loss is 1.47373, the accuracy is 0.492188
after 12900 iterationsthe loss is 1.39056, the accuracy is 0.578125
after 13000 iterationsthe loss is 1.28016, the accuracy is 0.601562
after 13100 iterationsthe loss is 1.27141, the accuracy is 0.570312
after 13200 iterationsthe loss is 1.27697, the accuracy is 0.609375
after 13300 iterationsthe loss is 1.30873, the accuracy is 0.476562
after 13400 iterationsthe loss is 1.31236, the accuracy is 0.554688
after 13500 iterationsthe loss is 1.41369, the accuracy is 0.484375
after 13600 iterationsthe loss is 1.26127, the accuracy is 0.617188
after 13700 iterationsthe loss is 1.30967, the accuracy is 0.601562
after 13800 iterationsthe loss is 1.35661, the accura

int